## Power generators

A power plant, with three available power generators, must provide at least of 4000 MW during the day and 2800 MW during the night. The generators can be activated to produce energy for a whole day or only half of it. The activation costs of the generators vary depending on the time of day, due the different cost of staff, as reported in the table.

Generator|Act. Cost Day|Act. Cost Night|Cost per MW
---------|-------------|---------------|-----------
Generator A|800|1200|4
Generator B|700|1000|6
Generator C|900|1400|7

The activation costs for the full day are the sum of those of day and night. The maximum production capacity are as follows.

Generator|Capacity (half day)|Capacity (full day)
---------|-------------------|-------------------
Generator A|2500|4000
Generator B|2000|3000
Generator C|3000|5000

Half day indicates that the power generators is on either by day or by night, while the full day denotes that the generator is active both day and night. 

Use mip to formulate and solve the problem of minimizing the total cost, guaranteeing the indicated energy requirements.

### Sets:
 * $I$ is the set of generators.

### Parameters:
 * $d^{d}$ is the demand of power during the day in MW
 * $d^{n}$ is the demand of power during the night in MW
 * $c_i$ is the cost per MW for generator $i \in I$
 * $f^d_i$ is the activation cost for generator i during the day
 * $f^n_i$ is the activation cost for generator i during the night
 * $k^{h}_i$ is the maximum capacity of generator i for half a day
 * $k^{f}_i$ is the maximum capacity of generator i for the full day

### Variables:
 * $x^d_i \ge 0$ is the amount of power delivered by generator i during the day
 * $x^n_i \ge 0$ is the amount of power delivered by generator i during the night
 * $y^d_i \in \{0, 1\}$ is equal to 1 if generator i is active only during the day, 0 otherwise
 * $y^n_i \in \{0, 1\}$ is equal to 1 if generator i is active only during the night, 0 otherwise
 * $z_i \in \{0, 1\}$ is equal to 1 if generator i is active both during the day and the night, 0
otherwise

### Model:
$$
\begin{array}{llll}
\min & \sum_{i \in I}{(c_i (x^d_i + x^n_i) + f^d_i (y^d_i + z_i) + f^n_i (y^n_i + z_i))} \\
\textrm{s.t.} & y^d_i + y^n_i + z_i \le 1&\qquad \forall i \in I \\
              & \sum_{i \in I}{x^d_i} \ge d^d &\qquad\\
              & \sum_{i \in I}{x^n_i} \ge d^n &\qquad\\
              & x^d_i \le k^h_i y^d_i + k^f_i z_i &\qquad \forall i \in I \\
              & x^n_i \le k^h_i y^n_i + k^f_i z_i &\qquad \forall i \in I \\
              & x^d_i + x^n_i \le k^h_i (1 - z_i) + k^f_i z_i &\qquad \forall i \in I \\
              & x^d_i, x^n_1 \in \mathbb R_+ &\qquad \forall i \in I \\
              & y^d_i, y^n_1, z_i \in \{0,1\} &\qquad \forall i \in I
\end{array}
$$

### Another formulation (we'll code this one):
$$
\begin{array}{llll}
\min & \sum_{i \in I}{(c_i (x^d_i + x^n_i) + f^d_i (y^d_i + z_i) + f^n_i (y^n_i + z_i))} \\
\textrm{s.t.} & y^d_i + y^n_i + z_i \le 1&\qquad \forall i \in I \\
              & \sum_{i \in I}{x^d_i} \ge d^d &\qquad\\
              & \sum_{i \in I}{x^n_i} \ge d^n &\qquad\\
              & x^d_i \le k^h_i y^d_i + \frac{k^f_i}{2} z_i &\qquad \forall i \in I \\
              & x^n_i \le k^h_i y^n_i + \frac{k^f_i}{2} z_i &\qquad \forall i \in I \\
              & x^d_i, x^n_1 \in \mathbb R_+ &\qquad \forall i \in I \\
              & y^d_i, y^n_1, z_i \in \{0,1\} &\qquad \forall i \in I
\end{array}
$$

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 427 kB 15.4 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.15.1
    Uninstalling cffi-1.15.1:
      Successfully uninstalled cffi-1.15.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 14.1 MB/s 


In [2]:
n_generators = 3
I = list(range(n_generators))

d_day = 4000 # [Mw]
d_night = 2800 # [Mw]

c = [4, 6, 7] # [$/Mw]

f_day = [800, 700, 900] # [$]
f_night = [1200, 1000, 1400] # [$]

k_half = [2500, 2000, 3000] # [Mw]
k_full = [4000, 3000, 5000] # [Mw]

In [6]:
import mip

m = mip.Model()

x_day = [m.add_var() for i in I]
x_night = [m.add_var() for i in I]

y_day = [m.add_var(var_type=mip.BINARY) for i in I]
y_night = [m.add_var(var_type=mip.BINARY) for i in I]
z = [m.add_var(var_type=mip.BINARY) for i in I]

m.add_constr(mip.xsum(x_day[i] for i in I) >= d_day)
m.add_constr(mip.xsum(x_night[i] for i in I) >= d_night)

for i in I:
    m.add_constr(y_day[i] + y_night[i] + z[i] <= 1)
    m.add_constr(x_day[i] <= k_half[i] * y_day[i] + 0.5 * k_full[i] * z[i])
    m.add_constr(x_night[i] <= k_half[i] * y_night[i] + 0.5 * k_full[i] * z[i])

m.objective = mip.minimize(mip.xsum(c[i]*(x_day[i] + x_night[i]) + f_day[i]*(y_day[i] + z[i]) + f_night[i]*(y_night[i] + z[i]) for i in I))

m.optimize()
print(m.objective_value)
print([x_day[i].x for i in I])
print([x_night[i].x for i in I])

37700.0
[2000.0, 2000.0, 0.0]
[2000.0, 0.0, 800.0]
